# Abstract

Learn about the data.

# Environment

In [ ]:
import sys
from pathlib import Path

In [ ]:
sys.path.insert(0, str(Path('..')))

In [ ]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from ipywidgets import interact
import numpy as np

In [ ]:
DATA_PATH = Path('.') / '..' / '..' / 'data'

# Library

In [ ]:
def moving_average(a, n=3):
    """Compute the moving average"""
    result = np.cumsum(a)
    result[n:] = result[n:] - result[:-n]
    return result[n-1:] / n

In [ ]:
def central_idxs_constraint(a, c=0.8, adj=0):
    """Return central indexes of regions meeting constraint"""
    central_idxs = []
    meets = np.abs(a) > c
    has_met = False
    for idx, met in enumerate(meets):
        if met:
            adj_idx = idx + adj
            if has_met:
                region.append(adj_idx)
            else:
                region = [adj_idx]
                has_met = True
        else:
            if has_met:
                central_idxs.append(int(np.mean(region)))
                has_met = False
    return central_idxs

# Main

In [ ]:
output_notebook()

## Read in the data

In [ ]:
data = {}
for fname in DATA_PATH.glob('*.npy'):
    data[fname.stem] = np.load(fname)

In [ ]:
jump = data['ubot']
idx_none = 0
idx_jumps = 1
idx_many_missed = 80

## Explore results

In [ ]:
mva_window = 10
jump_constraint = 0.7
jump_kernel = [-1., 1.]

In [ ]:
p_edge = figure(plot_width=950, tooltips=[("x", "$x"), ("value", "@y")])

In [ ]:
render_edge = p_edge.circle(range(40000), jump[0], size=3)
mva = moving_average(jump[0], n=mva_window)
mva_convolve = np.convolve(mva, jump_kernel, mode='valid')
jump_locations = central_idxs_constraint(mva_convolve, jump_constraint, adj=mva_window / 2 + 1)
print(jump_locations)
render_locs = p_edge.vbar(jump_locations, width=2, top=np.amax(jump[0]), bottom=np.amin(jump[0]), 
                         line_color='red')

In [ ]:
def update_p_edge(idx):
    mva = moving_average(jump[idx], n=mva_window)
    mva_convolve = np.convolve(mva, jump_kernel, mode='valid')
    jump_locations = central_idxs_constraint(mva_convolve, jump_constraint, adj=mva_window / 2 + 1)
    print(jump_locations)
    render_edge.data_source.data['y'] = jump[idx]
    render_locs.data_source.data['x'] = jump_locations
    render_locs.glyph.top = np.amax(jump[idx])
    render_locs.glyph.bottom = np.amin(jump[idx])
    push_notebook()

In [ ]:
show(p_edge, notebook_handle=True)

In [ ]:
interact(update_p_edge, idx=(0, data['ubot'].shape[0] - 1))

# Test & Utilities

## Plot a line

In [ ]:
p_nav = figure(plot_width=950, tooltips=[("x", "$x"), ("value", "@y")])

In [ ]:
render = p_nav.circle(range(40000), data['ubot'][0], size=4)

In [ ]:
def update_p_nav(idx):
    render.data_source.data['y'] = data['ubot'][idx]
    push_notebook()

In [ ]:
show(p_nav, notebook_handle=True)

In [ ]:
interact(update_p_nav, idx=(0, data['ubot'].shape[0] - 1))

## Moving Average

In [ ]:
mva_none = moving_average(jump[idx_none], n=10)

In [ ]:
p_mva_none = figure(plot_width=950)
p_mva_none.circle(range(len(mva_none)), mva_none, size=2)
show(p_mva_none)

In [ ]:
mva_jumps = moving_average(jump[idx_jumps], n=10)

In [ ]:
p_mva_jumps = figure(plot_width=950)
p_mva_jumps.circle(range(len(mva_jumps)), mva_jumps, size=2)
show(p_mva_jumps)

## Jump detection with basic convolution

In [ ]:
kernel = [-1., 1.]
# kernel = [1., -4., 1.]
# kernel = [4., -20., 4.]
# kernel = [-1., -2., 16, -2., -1.]

In [ ]:
c_none = np.convolve(jump[idx_none], kernel, mode='valid')

In [ ]:
p_convolve = figure(plot_width=950)

In [ ]:
p_convolve.circle(range(len(c_none)), c_none, size=2)

In [ ]:
show(p_convolve)

In [ ]:
c_jumps = np.convolve(mva_jumps, kernel, mode='valid')

In [ ]:
p_jumps = figure(plot_width=950)

In [ ]:
p_jumps.circle(range(len(c_jumps)), c_jumps, size=2)

In [ ]:
show(p_jumps)

In [ ]:
central_idxs = central_idxs_constraint(c_jumps, 0.7, adj=5+1)

In [ ]:
central_idxs